<a href="https://colab.research.google.com/github/karthiksagarN/Bart-FineTuned-Model-406M/blob/main/finetuning_bart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### TRAINING THE FINETUNED MODEL

In [ ]:
!pip install transformers[sentencepiece] datasets
!pip install transformers accelerate
!pip install --upgarde accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 61.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 59.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 91.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 29.0 MB/s eta 0:00:00
     ━━━━━━━━

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn")

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [ ]:
!unzip summarizer-data.zip

Archive:  summarizer-data.zip
  inflating: samsum-test.csv         
  inflating: samsum-train.csv        
  inflating: samsum-validation.csv   
   creating: samsum_dataset/
 extracting: samsum_dataset/dataset_dict.json  
   creating: samsum_dataset/test/
  inflating: samsum_dataset/test/data-00000-of-00001.arrow  
  inflating: samsum_dataset/test/dataset_info.json  
  inflating: samsum_dataset/test/state.json  
   creating: samsum_dataset/train/
  inflating: samsum_dataset/train/data-00000-of-00001.arrow  
  inflating: samsum_dataset/train/dataset_info.json  
  inflating: samsum_dataset/train/state.json  
   creating: samsum_dataset/validation/
  inflating: samsum_dataset/validation/data-00000-of-00001.arrow  
  inflating: samsum_dataset/validation/dataset_info.json  
  inflating: samsum_dataset/validation/state.json  


In [ ]:
from datasets import load_dataset, load_from_disk

dataset = load_from_disk("samsum_dataset")
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [ ]:
sample = dataset['test'][0]['dialogue']
label = dataset['test'][0]['summary']

def generate_summary(input , llm):
    input_prompt = f"""
                    Summarize the following conversation.
                    {input}
                    Summary:
                    """
    input_ids = tokenizer(sample, return_tensors='pt')
    tokenized_output = llm.generate(input_ids['input_ids'], min_length=30, max_length=200)
    output = tokenizer.decode(tokenized_output[0], skip_special_tokens=True)

    return output

output = generate_summary(sample, llm=model)
print("Sample")
print(sample)
print("--------------------")
print("Model Generated Summary:")
print(output)
print("Correct Summary:")
print(label)

Sample
Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye
--------------------
Model Generated Summary:
Hannah: Hey, do you have Betty's number? Amanda: Lemme check. Hannah: Ask Larry. Amanda: He called her last time we were at the park together.
Correct Summary:
Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.


In [ ]:
def tokenize_inputs(example):
    start_prompt = "Summarize the following conversation. \n\n"
    end_prompt = "\n\nSummary: "
    prompt = [start_prompt + dialogue + end_prompt for dialogue in example['dialogue']]
    example['input_ids'] = tokenizer (prompt, padding= 'max_length', truncation=True, return_tensors='pt').input_ids
    example['labels'] = tokenizer(example['summary'], padding="max_length", truncation=True, return_tensors= 'pt').input_ids

    return example

tokenizer.pad_token = tokenizer.eos_token
tokenized_datasets = dataset.map(tokenize_inputs, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['id', 'dialogue', 'summary'])
tokenized_datasets = tokenized_datasets.filter(lambda example, index : index%100==0, with_indices=True)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

Filter:   0%|          | 0/14732 [00:00<?, ? examples/s]

Filter:   0%|          | 0/819 [00:00<?, ? examples/s]

Filter:   0%|          | 0/818 [00:00<?, ? examples/s]

In [ ]:
print(tokenized_datasets['train'].shape)
print(tokenized_datasets['test'].shape)
print(tokenized_datasets['validation'].shape)

(148, 2)
(9, 2)
(9, 2)


In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./bart-samsum-finetuned",
    hub_model_id="karthiksagarn/bart-samsum-finetuned",
    learning_rate=1e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    auto_find_batch_size=True,
    # per_device_train_batch_size=8,  # Start with a smaller batch size
    warmup_steps=100,
    evaluation_strategy='epoch',
    logging_steps=10
)

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation']
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.223000,0.468666


Epoch,Training Loss,Validation Loss
1,0.119600,0.136231
2,0.094800,0.133365
3,0.073800,0.132568


TrainOutput(global_step=222, training_loss=0.09627780197439967, metrics={'train_runtime': 446.2115, 'train_samples_per_second': 0.995, 'train_steps_per_second': 0.498, 'total_flos': 1031541790605312.0, 'train_loss': 0.09627780197439967, 'epoch': 3.0})

In [ ]:
## PUSH OUR MODEL TO THE HUGGING FACE HUB
trainer.push_to_hub()

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


events.out.tfevents.1710793027.a9f004d34a77.603.0:   0%|          | 0.00/5.60k [00:00<?, ?B/s]

Upload 5 LFS files:   0%|          | 0/5 [00:00<?, ?it/s]

events.out.tfevents.1710793278.a9f004d34a77.603.1:   0%|          | 0.00/22.1k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.92k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

events.out.tfevents.1710793329.a9f004d34a77.603.2:   0%|          | 0.00/23.3k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/karthiksagarn/bart-samsum-finetuned/commit/d0895069aec0dcc39d710f90db64fdf8acf013d3', commit_message='End of training', commit_description='', oid='d0895069aec0dcc39d710f90db64fdf8acf013d3', pr_url=None, pr_revision=None, pr_num=None)

### Testing the Model

In [ ]:
Loaded_model = AutoModelForSeq2SeqLM.from_pretrained("karthiksagarn/bart-samsum-finetuned")

output = generate_summary(sample, llm=Loaded_model)
print("Sample")
print(sample)
print("--------------------")
print("Model Generated Summary:")
print(output)
print("--------------------")
print("Dataset Summary:")
print(label)

config.json:   0%|          | 0.00/1.66k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/358 [00:00<?, ?B/s]

Sample
Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye
--------------------
Model Generated Summary:
Hannah asks Betty's number and Amanda tries to find it for her. Betty doesn't know Larry very well, but he's very nice. She asks Larry to call Betty.
--------------------
Dataset Summary:
Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.


In [17]:
text = """
    The architecture of the model follows the conventional principles of Transformer and Attention, drawing from familiar sources like Common Crawl, Wikipedia, Books, along with some supplementary data. While much of the framework - pre-training, model structure, and data sources - resembles that of GPT-2, everything is magnified in scale, including the model's dimensions, dataset size, and training duration.
    It's the colossal scale of this model that predominantly drives its advantages.
"""
output = generate_summary(text, llm=Loaded_model)
print(output)

Hannah asks Betty's number and Amanda tries to find it for her. Betty doesn't know Larry very well, but he's very nice. She asks Larry to call Betty.


### Evaluation Metrics

In [25]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=aaca1db942da0903cff9cc09963733774d0a6d1950b0d77ac35f4b998935e68a
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [32]:
!pip install bert-score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.0 MB/s eta 0:00:00


In [33]:
from datasets import load_metric
from tqdm import tqdm
import pandas
import bert_score

In [19]:
device = "cpu"

In [31]:
from datasets import load_metric

# Load ROUGE metric
rouge_metric = load_metric("rouge")

# Function to generate summaries
def generate_summary(input, model):
    input_prompt = f"""
                    Summarize the following conversation.
                    {input}
                    Summary:
                    """
    input_ids = tokenizer(input_prompt, return_tensors='pt').input_ids.to(model.device)
    tokenized_output = model.generate(input_ids, min_length=30, max_length=200)
    output = tokenizer.decode(tokenized_output[0], skip_special_tokens=True)

    return output

# Function to compute ROUGE scores
def compute_rouge_scores(dataset, model):
    predictions = []
    references = []

    for example in dataset:
        dialogue = example['dialogue']
        summary = example['summary']

        # Generate summary using the model
        generated_summary = generate_summary(dialogue, model)

        # Append to predictions and references
        predictions.append(generated_summary)
        references.append(summary)

    # Compute ROUGE scores
    rouge_output = rouge_metric.compute(predictions=predictions, references=references)

    return rouge_output

# Compute ROUGE scores for validation dataset
rouge_scores = compute_rouge_scores(dataset['validation'], model)

# Print ROUGE scores
print("ROUGE Scores:")
print(rouge_scores)


/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


ROUGE Scores:
{'rouge1': AggregateScore(low=Score(precision=0.29231890922960946, recall=0.5755340591660789, fmeasure=0.36452270126617037), mid=Score(precision=0.3012174062385683, recall=0.5880590685871268, fmeasure=0.3721959667623731), high=Score(precision=0.31083014490046085, recall=0.6010687364120285, fmeasure=0.38112972171463944)), 'rouge2': AggregateScore(low=Score(precision=0.11850674794550704, recall=0.2418211034418429, fmeasure=0.14808409500530648), mid=Score(precision=0.12516700754290633, recall=0.2545193627452707, fmeasure=0.15550558913151516), high=Score(precision=0.13212297153452818, recall=0.26821321492196054, fmeasure=0.16319578693000844)), 'rougeL': AggregateScore(low=Score(precision=0.21821073377249478, recall=0.4433629225420538, fmeasure=0.2744300617502658), mid=Score(precision=0.2250875126650962, recall=0.454741311774031, fmeasure=0.2810128477652416), high=Score(precision=0.23278079521177517, recall=0.4679046460070841, fmeasure=0.2886331732350875)), 'rougeLsum': Aggreg

In [35]:
from bert_score import score

# Function to compute BERTScore
def compute_bert_score(dataset, model):
    predictions = []
    references = []

    for example in dataset:
        dialogue = example['dialogue']
        summary = example['summary']

        # Generate summary using the model
        generated_summary = generate_summary(dialogue, model)

        # Append to predictions and references
        predictions.append(generated_summary)
        references.append(summary)

    # Compute BERTScore
    precision, recall, f1 = score(predictions, references, lang='en', model_type='bert-base-uncased')

    return precision, recall, f1

# Compute BERTScore for validation dataset
precision, recall, f1 = compute_bert_score(dataset['validation'], model)

# Print BERTScore
print("BERTScore:")
print("Precision:", precision.mean().item())
print("Recall:", recall.mean().item())
print("F1:", f1.mean().item())


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BERTScore:
Precision: 0.6054495573043823
Recall: 0.6918860077857971
F1: 0.6425597667694092


In [36]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
